# SDP Data Flow Graph Generation and Analysis

(Last tested with IPython 4.0.0 using Python 2.7 and GraphViz 2.36 - If you have trouble, check version compatibility)

In [ ]:
""" These first few lines below import the IPython definitions and methods that we will use. 
Please Refer to ipython_api.py for the implementation """
from ipywidgets import interact, interact_manual, fixed, SelectMultiple
from IPython.display import display, SVG

from pipeline import Pipeline, flowsToDot
from parameter_definitions import *
from implementation import Implementation as imp, PipelineConfig
%matplotlib inline

# Make a configuration
cfg = PipelineConfig(telescope=Telescopes.SKA1_Mid,
                     band=Bands.Mid1,
                     pipeline=Pipelines.ICAL,
                     max_baseline='default',
                     Nf_max='default',
                     blcoal=True,
                     on_the_fly=False)

In [ ]:
# Calculate optimised parameters
tp = imp.calc_tel_params(cfg)
(tsnap_opt, nfacet_opt) = imp.find_optimal_Tsnap_Nfacet(tp)
tp = imp.calc_tel_params(cfg, adjusts={ 'Nfacet': nfacet_opt, 'Tsnap': tsnap_opt })

# Generate DOT and render
Trun = tp.Tobs # s, guessed (should depend on pipeline)
Rhardware = 17.8 * Constants.tera # FLOP/s, from cost model
print "Assuming a pipeline run time of %02d:%02d:" % (Trun / 60, Trun % 60)
dot = flowsToDot(Pipeline(tp).create_pipeline(), Trun, Rhardware,
                 graph_attr={'size':'10,100'},
                 node_attr={'shape':'box', 'style':'filled','color':'lightgrey'})
display(SVG(dot._repr_svg_()))

### Reading Guide

| Label | Example Value | Explanation |
| ------------- | ------ | | 
| Grid          |        | | 
| Major Loop:   | 10 x 1 | Runs once per major loop (10x)|
| Facet:        | 16 x 1 | Runs once per facet (16x)|
| Polarisation: | 4 x 1  | Runs once per polarisation (4x)|
| Time: | 444 x 48.5712 s | Runs once per 48.5 s of data (444x) |
| Frequency: | 40681 x 48.3103 ch | Run per 48.3 channels of input data (40681 x) |
| Baseline: | 12 x 11871.1 | Run per each of the 12 baseline bins, containing at maximum 11871.1 baselines |
| Tasks: | 3792528 1/s | Task rate (assuming even task distribution over time) |
| Time: | 9.1e-06 s/task | Run time for individual task (assuming no aggregation and given hardware speed) |
| FLOPs: | 0.61 POP/s | Total floating point operation rate contribution (assuming even task distribution over time) | 
| Output: | 124971.46 TB/s | Total data output (assuming even task distribution over time) |

## The graph can be exported

In [ ]:
# Export, if desired
dot.format='pdf' # png, svg, ...
dot.render('%s_%s_dataflow' % (cfg.pipeline, cfg.band))

## We can also look at data flow properties symbolically

This is especially useful because they should match the product formulas pretty much exactly. When a test case complains about a mismatch, this will be useful to track down differences.

In [ ]:
from IPython.display import display, Math, Latex, HTML
from sympy import latex, Lambda

# Get symbolised parameters
tp = imp.calc_tel_params(cfg, symbolify='product')
flows = Pipeline(tp).create_imaging()
for flow in sorted(flows, key=lambda f: f.name):
    display(Math(latex(Symbol('R_{flop,'+flow.name+'}')) + ' = ' + latex(flow.cost('compute')/tp.Tobs)))